# Generate automatic documentation for individual project

In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [0]:
PROJECT_KEY = 'TIM_HEALTH_DATA'
DATASET_ID = 'Apple_Health_daily_summaries'

In [0]:
client = dataiku.api_client()
project = client.get_project(PROJECT_KEY)
dataset_handle = project.get_dataset(DATASET_ID)

In [0]:
dataset_handle

In [0]:
# # starting it:
# curl 'https://10.0.1.37:8443/dip/api/flow/ai-tools/start-dataset-explanation' \
#   -H 'Accept: application/json, text/plain, */*' \
#   -H 'Accept-Language: en-US,en;q=0.9' \
#   -H 'Cache-Control: no-cache' \
#   -H 'Connection: keep-alive' \
#   -H 'Content-Type: application/x-www-form-urlencoded;charset=UTF-8' \
#   -b '__wt1vic=be9460313558f3c; dss_identity_token_66cebbb596cf6380a393ae2370bdc169=3LzWTLEHzoQqCpyjC9Da95jOmIc9Qz9MGx5rZLKYD2DdhqRHS06J0a2Nyz2Uawr1; dss_access_token_66cebbb596cf6380a393ae2370bdc169=gh7O5hsbZsWWxdXSxE0TlCfi1d5EuwAZnawtOcXcVct16M6QG46icYOFVF5LbUKP; dss_xsrf_token_66cebbb596cf6380a393ae2370bdc169=ea04da98f4318e80a9f0f76873bd645b6ff22529971d06cffa9b15a40b9140d2; X-DKU-Flask-User=HrzuULlbIAUihAzQdYWvDzCbFmWvYhsr; dss_identity_token_0838b03a94805a04c768295fce7951f9=4PKWWvbcTkf59xRlhcPMJln1AVj5SSDlmwpBOygK6MH25eZq8BRoi7lDFcre7uh2; dss_access_token_0838b03a94805a04c768295fce7951f9=r7c2WXxYa5LfRjuCPQAR88dPdmF8GKHab0k7Y6jtaViiK1pjYixqLOWMSvqb1vIP; dss_xsrf_token_0838b03a94805a04c768295fce7951f9=cfd4221a9d12a8c4e48e731561d489600df381a160cb61a4b94ff9bd183a7a87; _xsrf=2|4845a794|ee4da3444be018105db1bd3658f74076|1759516840; __wt1vpc=dipInstanceId%3Ddevl1-timhonker-504caa6fdebbbe241401068441abb5e6-2d17c61e4416e1c4d34957423a3f9671%26installId%3DO6WDoaZUEj3YXWcAceKznT4w%26dssLicenseKind%3DDATAIKU_INTERNAL%26bkdDistrib%3Dubuntu%26bkdDistribVersion%3D22.04%26deploymentMode%3DCUSTOM%26nodeType%3Ddesign%26enabledFeatureFlags%3DGOVERN_INTEGRATION%252CllmFineTuningSupport%252Cstreaming%252CllmFineTuningExperimentalSupport%252CtestingWithDss%252CaiExplain%252CaiPrepare%252CllmCoreSupport%252CcodeAssistant%252ChomepageRedesign%252Cstory%252CllmEvaluation%26isAutomation%3Dfalse%26dssVersion%3D14.2.0%26vdssUser%3D92668751; __wt1sic=11a6e5d15553184; __wt1spc=currentState%3Dprojects.project.datasets.dataset.explore' \
#   -H 'Origin: https://10.0.1.37:8443' \
#   -H 'Pragma: no-cache' \
#   -H 'Referer: https://10.0.1.37:8443/projects/TIM_HEALTH_DATA/datasets/Apple_Health_daily_summaries/explore/' \
#   -H 'Sec-Fetch-Dest: empty' \
#   -H 'Sec-Fetch-Mode: cors' \
#   -H 'Sec-Fetch-Site: same-origin' \
#   -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36' \
#   -H 'X-XSRF-TOKEN: cfd4221a9d12a8c4e48e731561d489600df381a160cb61a4b94ff9bd183a7a87' \
#   -H 'sec-ch-ua: "Google Chrome";v="141", "Not?A_Brand";v="8", "Chromium";v="141"' \
#   -H 'sec-ch-ua-mobile: ?0' \
#   -H 'sec-ch-ua-platform: "macOS"' \
#   -H 'version-id: 14.2.0' \
#   --data-raw 'projectKey=TIM_HEALTH_DATA&datasetName=Apple_Health_daily_summaries&explanationOptions=%7B%22language%22%3A%22english%22%7D' \
#   --insecure

# # response has JobID, call with futures
# {
#     "hasResult": false,
#     "aborted": false,
#     "alive": true,
#     "startTime": 1761573091025,
#     "runningTime": 1,
#     "unknown": false,
#     "jobId": "ILerZ46k"
# }


# # keep checking it until it is finished:
# curl 'https://10.0.1.37:8443/dip/api/futures/get-update?futureId=ILerZ46k' \
#   -H 'Accept: application/json, text/plain, */*' \
#   -H 'Accept-Language: en-US,en;q=0.9' \
#   -H 'Cache-Control: no-cache' \
#   -H 'Connection: keep-alive' \
#   -b '__wt1vic=be9460313558f3c; dss_identity_token_66cebbb596cf6380a393ae2370bdc169=3LzWTLEHzoQqCpyjC9Da95jOmIc9Qz9MGx5rZLKYD2DdhqRHS06J0a2Nyz2Uawr1; dss_access_token_66cebbb596cf6380a393ae2370bdc169=gh7O5hsbZsWWxdXSxE0TlCfi1d5EuwAZnawtOcXcVct16M6QG46icYOFVF5LbUKP; dss_xsrf_token_66cebbb596cf6380a393ae2370bdc169=ea04da98f4318e80a9f0f76873bd645b6ff22529971d06cffa9b15a40b9140d2; X-DKU-Flask-User=HrzuULlbIAUihAzQdYWvDzCbFmWvYhsr; dss_identity_token_0838b03a94805a04c768295fce7951f9=4PKWWvbcTkf59xRlhcPMJln1AVj5SSDlmwpBOygK6MH25eZq8BRoi7lDFcre7uh2; dss_access_token_0838b03a94805a04c768295fce7951f9=r7c2WXxYa5LfRjuCPQAR88dPdmF8GKHab0k7Y6jtaViiK1pjYixqLOWMSvqb1vIP; dss_xsrf_token_0838b03a94805a04c768295fce7951f9=cfd4221a9d12a8c4e48e731561d489600df381a160cb61a4b94ff9bd183a7a87; _xsrf=2|4845a794|ee4da3444be018105db1bd3658f74076|1759516840; __wt1vpc=dipInstanceId%3Ddevl1-timhonker-504caa6fdebbbe241401068441abb5e6-2d17c61e4416e1c4d34957423a3f9671%26installId%3DO6WDoaZUEj3YXWcAceKznT4w%26dssLicenseKind%3DDATAIKU_INTERNAL%26bkdDistrib%3Dubuntu%26bkdDistribVersion%3D22.04%26deploymentMode%3DCUSTOM%26nodeType%3Ddesign%26enabledFeatureFlags%3DGOVERN_INTEGRATION%252CllmFineTuningSupport%252Cstreaming%252CllmFineTuningExperimentalSupport%252CtestingWithDss%252CaiExplain%252CaiPrepare%252CllmCoreSupport%252CcodeAssistant%252ChomepageRedesign%252Cstory%252CllmEvaluation%26isAutomation%3Dfalse%26dssVersion%3D14.2.0%26vdssUser%3D92668751; __wt1sic=11a6e5d15553184; __wt1spc=currentState%3Dprojects.project.datasets.dataset.explore' \
#   -H 'Pragma: no-cache' \
#   -H 'Referer: https://10.0.1.37:8443/projects/TIM_HEALTH_DATA/datasets/Apple_Health_daily_summaries/explore/' \
#   -H 'Sec-Fetch-Dest: empty' \
#   -H 'Sec-Fetch-Mode: cors' \
#   -H 'Sec-Fetch-Site: same-origin' \
#   -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36' \
#   -H 'X-XSRF-TOKEN: cfd4221a9d12a8c4e48e731561d489600df381a160cb61a4b94ff9bd183a7a87' \
#   -H 'sec-ch-ua: "Google Chrome";v="141", "Not?A_Brand";v="8", "Chromium";v="141"' \
#   -H 'sec-ch-ua-mobile: ?0' \
#   -H 'sec-ch-ua-platform: "macOS"' \
#   -H 'version-id: 14.2.0' \
#   --insecure

# # still waiting response:
# {
#     "hasResult": false,
#     "aborted": false,
#     "alive": true,
#     "startTime": 1761573091025,
#     "runningTime": 27,
#     "unknown": false,
#     "jobId": "ILerZ46k",
#     "progress": {
#         "states": [
#             {
#                 "name": "Computing",
#                 "unit": "NONE",
#                 "target": 5.0,
#                 "cur": 1.0,
#                 "important": false,
#                 "depth": 0,
#                 "startTimestamp": 1761573091030,
#                 "msSinceStart": 22
#             },
#             {
#                 "name": "Building sample",
#                 "unit": "NONE",
#                 "target": -1.0,
#                 "cur": 0.0,
#                 "important": false,
#                 "depth": 0,
#                 "startTimestamp": 1761573091042,
#                 "msSinceStart": 10
#             }
#         ]
#     }
# }


# # ready response:
# {
#     "hasResult": true,
#     "aborted": false,
#     "alive": false,
#     "startTime": 0,
#     "runningTime": 0,
#     "unknown": false,
#     "result": {
#         "ok": true,
#         "requestId": "5d412cc2-3166-4acc-bdc3-2eb4bb6abdda",
#         "shortDesc": "Apple Health daily summaries with health metrics and activity data.",
#         "longDesc": "The Apple Health daily summaries dataset provides a comprehensive overview of various health and activity metrics collected on a daily basis. This dataset is particularly useful for individuals or researchers interested in tracking health trends, analyzing physical activity, and understanding personal health metrics over time. It includes a wide range of data points such as active energy expenditure, blood pressure readings, body composition metrics, and detailed sleep analysis. Additionally, it tracks daily activities like step count, flights climbed, and walking or running distances. Unique features of this dataset include detailed sleep analysis with metrics like core, deep, and REM sleep, as well as sexual activity tracking. The dataset can be used for personal health monitoring, fitness tracking, or even in research studies focusing on health and wellness. By analyzing this data, users can gain insights into their daily health patterns, identify areas for improvement, and make informed decisions about their lifestyle and health management.",
#         "colDesc": {
#             "Active Energy (kcal)": "Calories burned through activity, measures daily energy expenditure.",
#             "Blood Pressure [Diastolic] (mmHg)": "Diastolic blood pressure, important for heart health.",
#             "Blood Pressure [Systolic] (mmHg)": "Systolic blood pressure, indicates cardiovascular health.",
#             "Body Fat Percentage (%)": "Body fat percentage, assesses body composition.",
#             "Body Mass Index (count)": "BMI, evaluates body weight relative to height.",
#             "Date": "The date of the recorded data, key for time series analysis.",
#             "Flights Climbed (count)": "Number of flights of stairs climbed, measures physical activity.",
#             "Headphone Audio Exposure (dBASPL)": "Audio exposure level, monitors hearing health.",
#             "Heart Rate [Avg] (bpm)": "Average heart rate, reflects overall heart activity.",
#             "Heart Rate [Max] (bpm)": "Maximum heart rate, indicates cardiovascular exertion.",
#             "Lean Body Mass (lb)": "Lean body mass, part of body composition analysis.",
#             "Respiratory Rate (count/min)": "Breathing rate, important for respiratory health.",
#             "Resting Heart Rate (bpm)": "Resting heart rate, indicates cardiovascular fitness.",
#             "Sleep Analysis [Asleep] (hr)": "Total hours asleep, measures sleep duration.",
#             "Sleep Analysis [Awake] (hr)": "Awake hours during sleep, indicates sleep disturbances.",
#             "Sleep Analysis [Core] (hr)": "Core sleep hours, part of sleep quality analysis.",
#             "Sleep Analysis [Deep] (hr)": "Deep sleep hours, crucial for restorative sleep.",
#             "Sleep Analysis [In Bed] (hr)": "Total hours in bed, indicates sleep opportunity.",
#             "Sleep Analysis [REM] (hr)": "REM sleep hours, important for cognitive health.",
#             "Step Count (steps)": "Total steps taken, measures daily activity level.",
#             "Walking + Running Distance (mi)": "Distance covered walking/running, tracks physical activity.",
#             "Water (fl. oz.)": "Water intake, important for hydration monitoring.",
#             "Weight/Body Mass (lb)": "Body weight, key for tracking weight changes.",
#             "age_days": "Age in days, useful for age-related analysis."
#         }
#     }
# }


# # this GET step may not be necessary?
# # curl 'https://10.0.1.37:8443/static/dataiku/ai-dataset-descriptions/generate-documentation-approval/generate-documentation-approval.component.html' \
# #   -H 'Accept: application/json, text/plain, */*' \
# #   -H 'Accept-Language: en-US,en;q=0.9' \
# #   -H 'Cache-Control: no-cache' \
# #   -H 'Connection: keep-alive' \
# #   -b '__wt1vic=be9460313558f3c; dss_identity_token_66cebbb596cf6380a393ae2370bdc169=3LzWTLEHzoQqCpyjC9Da95jOmIc9Qz9MGx5rZLKYD2DdhqRHS06J0a2Nyz2Uawr1; dss_access_token_66cebbb596cf6380a393ae2370bdc169=gh7O5hsbZsWWxdXSxE0TlCfi1d5EuwAZnawtOcXcVct16M6QG46icYOFVF5LbUKP; dss_xsrf_token_66cebbb596cf6380a393ae2370bdc169=ea04da98f4318e80a9f0f76873bd645b6ff22529971d06cffa9b15a40b9140d2; X-DKU-Flask-User=HrzuULlbIAUihAzQdYWvDzCbFmWvYhsr; dss_identity_token_0838b03a94805a04c768295fce7951f9=4PKWWvbcTkf59xRlhcPMJln1AVj5SSDlmwpBOygK6MH25eZq8BRoi7lDFcre7uh2; dss_access_token_0838b03a94805a04c768295fce7951f9=r7c2WXxYa5LfRjuCPQAR88dPdmF8GKHab0k7Y6jtaViiK1pjYixqLOWMSvqb1vIP; dss_xsrf_token_0838b03a94805a04c768295fce7951f9=cfd4221a9d12a8c4e48e731561d489600df381a160cb61a4b94ff9bd183a7a87; _xsrf=2|4845a794|ee4da3444be018105db1bd3658f74076|1759516840; __wt1vpc=dipInstanceId%3Ddevl1-timhonker-504caa6fdebbbe241401068441abb5e6-2d17c61e4416e1c4d34957423a3f9671%26installId%3DO6WDoaZUEj3YXWcAceKznT4w%26dssLicenseKind%3DDATAIKU_INTERNAL%26bkdDistrib%3Dubuntu%26bkdDistribVersion%3D22.04%26deploymentMode%3DCUSTOM%26nodeType%3Ddesign%26enabledFeatureFlags%3DGOVERN_INTEGRATION%252CllmFineTuningSupport%252Cstreaming%252CllmFineTuningExperimentalSupport%252CtestingWithDss%252CaiExplain%252CaiPrepare%252CllmCoreSupport%252CcodeAssistant%252ChomepageRedesign%252Cstory%252CllmEvaluation%26isAutomation%3Dfalse%26dssVersion%3D14.2.0%26vdssUser%3D92668751; __wt1sic=11a6e5d15553184; __wt1spc=currentState%3Dprojects.project.datasets.dataset.explore' \
# #   -H 'Pragma: no-cache' \
# #   -H 'Referer: https://10.0.1.37:8443/projects/TIM_HEALTH_DATA/datasets/Apple_Health_daily_summaries/explore/' \
# #   -H 'Sec-Fetch-Dest: empty' \
# #   -H 'Sec-Fetch-Mode: cors' \
# #   -H 'Sec-Fetch-Site: same-origin' \
# #   -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36' \
# #   -H 'sec-ch-ua: "Google Chrome";v="141", "Not?A_Brand";v="8", "Chromium";v="141"' \
# #   -H 'sec-ch-ua-mobile: ?0' \
# #   -H 'sec-ch-ua-platform: "macOS"' \
# #   --insecure

# # setting descriptions
# curl 'https://10.0.1.37:8443/dip/api/datasets/set-dataset-descriptions' \
#   -H 'Accept: application/json, text/plain, */*' \
#   -H 'Accept-Language: en-US,en;q=0.9' \
#   -H 'Cache-Control: no-cache' \
#   -H 'Connection: keep-alive' \
#   -H 'Content-Type: application/x-www-form-urlencoded;charset=UTF-8' \
#   -b '__wt1vic=be9460313558f3c; dss_identity_token_66cebbb596cf6380a393ae2370bdc169=3LzWTLEHzoQqCpyjC9Da95jOmIc9Qz9MGx5rZLKYD2DdhqRHS06J0a2Nyz2Uawr1; dss_access_token_66cebbb596cf6380a393ae2370bdc169=gh7O5hsbZsWWxdXSxE0TlCfi1d5EuwAZnawtOcXcVct16M6QG46icYOFVF5LbUKP; dss_xsrf_token_66cebbb596cf6380a393ae2370bdc169=ea04da98f4318e80a9f0f76873bd645b6ff22529971d06cffa9b15a40b9140d2; X-DKU-Flask-User=HrzuULlbIAUihAzQdYWvDzCbFmWvYhsr; dss_identity_token_0838b03a94805a04c768295fce7951f9=4PKWWvbcTkf59xRlhcPMJln1AVj5SSDlmwpBOygK6MH25eZq8BRoi7lDFcre7uh2; dss_access_token_0838b03a94805a04c768295fce7951f9=r7c2WXxYa5LfRjuCPQAR88dPdmF8GKHab0k7Y6jtaViiK1pjYixqLOWMSvqb1vIP; dss_xsrf_token_0838b03a94805a04c768295fce7951f9=cfd4221a9d12a8c4e48e731561d489600df381a160cb61a4b94ff9bd183a7a87; _xsrf=2|4845a794|ee4da3444be018105db1bd3658f74076|1759516840; __wt1vpc=dipInstanceId%3Ddevl1-timhonker-504caa6fdebbbe241401068441abb5e6-2d17c61e4416e1c4d34957423a3f9671%26installId%3DO6WDoaZUEj3YXWcAceKznT4w%26dssLicenseKind%3DDATAIKU_INTERNAL%26bkdDistrib%3Dubuntu%26bkdDistribVersion%3D22.04%26deploymentMode%3DCUSTOM%26nodeType%3Ddesign%26enabledFeatureFlags%3DGOVERN_INTEGRATION%252CllmFineTuningSupport%252Cstreaming%252CllmFineTuningExperimentalSupport%252CtestingWithDss%252CaiExplain%252CaiPrepare%252CllmCoreSupport%252CcodeAssistant%252ChomepageRedesign%252Cstory%252CllmEvaluation%26isAutomation%3Dfalse%26dssVersion%3D14.2.0%26vdssUser%3D92668751; __wt1sic=11a6e5d15553184; __wt1spc=currentState%3Dprojects.project.datasets.dataset.explore' \
#   -H 'Origin: https://10.0.1.37:8443' \
#   -H 'Pragma: no-cache' \
#   -H 'Referer: https://10.0.1.37:8443/projects/TIM_HEALTH_DATA/datasets/Apple_Health_daily_summaries/explore/' \
#   -H 'Sec-Fetch-Dest: empty' \
#   -H 'Sec-Fetch-Mode: cors' \
#   -H 'Sec-Fetch-Site: same-origin' \
#   -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36' \
#   -H 'X-XSRF-TOKEN: cfd4221a9d12a8c4e48e731561d489600df381a160cb61a4b94ff9bd183a7a87' \
#   -H 'sec-ch-ua: "Google Chrome";v="141", "Not?A_Brand";v="8", "Chromium";v="141"' \
#   -H 'sec-ch-ua-mobile: ?0' \
#   -H 'sec-ch-ua-platform: "macOS"' \
#   -H 'version-id: 14.2.0' \
#   --data-raw 'projectKey=TIM_HEALTH_DATA&datasetName=Apple_Health_daily_summaries&shortDesc=Apple%20Health%20daily%20summaries%20with%20health%20metrics%20and%20activity%20data.&longDesc=The%20Apple%20Health%20daily%20summaries%20dataset%20provides%20a%20comprehensive%20overview%20of%20various%20health%20and%20activity%20metrics%20collected%20on%20a%20daily%20basis.%20This%20dataset%20is%20particularly%20useful%20for%20individuals%20or%20researchers%20interested%20in%20tracking%20health%20trends%2C%20analyzing%20physical%20activity%2C%20and%20understanding%20personal%20health%20metrics%20over%20time.%20It%20includes%20a%20wide%20range%20of%20data%20points%20such%20as%20active%20energy%20expenditure%2C%20blood%20pressure%20readings%2C%20body%20composition%20metrics%2C%20and%20detailed%20sleep%20analysis.%20Additionally%2C%20it%20tracks%20daily%20activities%20like%20step%20count%2C%20flights%20climbed%2C%20and%20walking%20or%20running%20distances.%20Unique%20features%20of%20this%20dataset%20include%20detailed%20sleep%20analysis%20with%20metrics%20like%20core%2C%20deep%2C%20and%20REM%20sleep%2C%20as%20well%20as%20sexual%20activity%20tracking.%20The%20dataset%20can%20be%20used%20for%20personal%20health%20monitoring%2C%20fitness%20tracking%2C%20or%20even%20in%20research%20studies%20focusing%20on%20health%20and%20wellness.%20By%20analyzing%20this%20data%2C%20users%20can%20gain%20insights%20into%20their%20daily%20health%20patterns%2C%20identify%20areas%20for%20improvement%2C%20and%20make%20informed%20decisions%20about%20their%20lifestyle%20and%20health%20management.&colDesc=%7B%22Active%20Energy%20(kcal)%22%3A%22Calories%20burned%20through%20activity%2C%20measures%20daily%20energy%20expenditure.%22%2C%22Blood%20Pressure%20%5BDiastolic%5D%20(mmHg)%22%3A%22Diastolic%20blood%20pressure%2C%20important%20for%20heart%20health.%22%2C%22Blood%20Pressure%20%5BSystolic%5D%20(mmHg)%22%3A%22Systolic%20blood%20pressure%2C%20indicates%20cardiovascular%20health.%22%2C%22Body%20Fat%20Percentage%20(%25)%22%3A%22Body%20fat%20percentage%2C%20assesses%20body%20composition.%22%2C%22Body%20Mass%20Index%20(count)%22%3A%22BMI%2C%20evaluates%20body%20weight%20relative%20to%20height.%22%2C%22Date%22%3A%22The%20date%20of%20the%20recorded%20data%2C%20key%20for%20time%20series%20analysis.%22%2C%22Flights%20Climbed%20(count)%22%3A%22Number%20of%20flights%20of%20stairs%20climbed%2C%20measures%20physical%20activity.%22%2C%22Headphone%20Audio%20Exposure%20(dBASPL)%22%3A%22Audio%20exposure%20level%2C%20monitors%20hearing%20health.%22%2C%22Heart%20Rate%20%5BAvg%5D%20(bpm)%22%3A%22Average%20heart%20rate%2C%20reflects%20overall%20heart%20activity.%22%2C%22Heart%20Rate%20%5BMax%5D%20(bpm)%22%3A%22Maximum%20heart%20rate%2C%20indicates%20cardiovascular%20exertion.%22%2C%22Lean%20Body%20Mass%20(lb)%22%3A%22Lean%20body%20mass%2C%20part%20of%20body%20composition%20analysis.%22%2C%22Respiratory%20Rate%20(count%2Fmin)%22%3A%22Breathing%20rate%2C%20important%20for%20respiratory%20health.%22%2C%22Resting%20Heart%20Rate%20(bpm)%22%3A%22Resting%20heart%20rate%2C%20indicates%20cardiovascular%20fitness.%22%2C%22Sexual%20Activity%20%5BProtection%20Not%20Used%5D%20(times)%22%3A%22Unprotected%20sexual%20activity%20frequency%2C%20health%20risk.%22%2C%22Sexual%20Activity%20%5BProtection%20Used%5D%20(times)%22%3A%22Protected%20sexual%20activity%20frequency%2C%20health%20behavior.%22%2C%22Sexual%20Activity%20%5BUnspecified%5D%20(times)%22%3A%22Frequency%20of%20sexual%20activity%2C%20general%20health%20indicator.%22%2C%22Sleep%20Analysis%20%5BAsleep%5D%20(hr)%22%3A%22Total%20hours%20asleep%2C%20measures%20sleep%20duration.%22%2C%22Sleep%20Analysis%20%5BAwake%5D%20(hr)%22%3A%22Awake%20hours%20during%20sleep%2C%20indicates%20sleep%20disturbances.%22%2C%22Sleep%20Analysis%20%5BCore%5D%20(hr)%22%3A%22Core%20sleep%20hours%2C%20part%20of%20sleep%20quality%20analysis.%22%2C%22Sleep%20Analysis%20%5BDeep%5D%20(hr)%22%3A%22Deep%20sleep%20hours%2C%20crucial%20for%20restorative%20sleep.%22%2C%22Sleep%20Analysis%20%5BIn%20Bed%5D%20(hr)%22%3A%22Total%20hours%20in%20bed%2C%20indicates%20sleep%20opportunity.%22%2C%22Sleep%20Analysis%20%5BREM%5D%20(hr)%22%3A%22REM%20sleep%20hours%2C%20important%20for%20cognitive%20health.%22%2C%22Step%20Count%20(steps)%22%3A%22Total%20steps%20taken%2C%20measures%20daily%20activity%20level.%22%2C%22Walking%20%2B%20Running%20Distance%20(mi)%22%3A%22Distance%20covered%20walking%2Frunning%2C%20tracks%20physical%20activity.%22%2C%22Water%20(fl.%20oz.)%22%3A%22Water%20intake%2C%20important%20for%20hydration%20monitoring.%22%2C%22Weight%2FBody%20Mass%20(lb)%22%3A%22Body%20weight%2C%20key%20for%20tracking%20weight%20changes.%22%2C%22age_days%22%3A%22Age%20in%20days%2C%20useful%20for%20age-related%20analysis.%22%7D' \
#   --insecure


# https://developer.dataiku.com/latest/api-reference/python/datasets.html#dataikuapi.dss.dataset.DSSDataset.generate_ai_description

In [0]:
# this blocks execution, no futures
dataset_handle.generate_ai_description(save_description=True)

In [0]:
md = dataset_handle.get_metadata()

